In [1]:
import numpy as np
import json
from bottle import Bottle, request, response, run, template, static_file, redirect

app = Bottle()

In [2]:
#generating S-box, following C program from link below
#https://en.wikipedia.org/wiki/Rijndael_S-box

def rotate(x,shift):
    #Rotate 8-bit integer `x` to the left by `shift` positions.
    return ((x << shift) | (x >> (8 - shift))) & 0xFF

def create_SBOX():  
    SBOX = [0] * 256
    p = 1
    q = 1
    
    while True:
        #multiply p by 3
        p ^= (p << 1) ^ (0x1B if p & 0x80 else 0)
        p &= 0xFF
        
        #divide q by 3 (equals multiplication by 0xf6)
        q ^= (q << 1) & 0xFF
        q ^= (q << 2) & 0xFF
        q ^= (q << 4) & 0xFF
        q ^= (0x09 if q & 0x80 else 0)
        q &= 0xFF  
        
        #compute the affine transformation
        xformed = q ^ rotate(q, 1) ^ rotate(q, 2) ^ rotate(q, 3) ^ rotate(q, 4)
        
        SBOX[p] = xformed ^ 0x63
        
        if(p == 1):
            break
        
    #special case for zero
    SBOX[0] = 0x63
    return SBOX


In [3]:
strings = {}  # List to store input, encryption, and unique IDs
next_id = 1  # Unique ID for each request

# AES constants and S-box (for AES-128)
SBOX = create_SBOX()

Nb = 4  # Block size for AES (in 32-bit words)
Nk = 4  # Key length (in 32-bit words) for AES-128
Nr = 10 # Number of rounds for AES-128

In [4]:
def key_expansion_gov(key):   #Expand the cipher key into the key schedule
    
    def sub_word(word):   #Apply the AES S-box to each byte in a word
        #return bytes(SBOX[b] for b in word)
        return [SBOX[byte] for byte in word]


    def rot_word(word):   #Perform a cyclic permutation on a 4-byte word
        return word[1:] + word[:1]


    RCON = [
        [0x01, 0x00, 0x00, 0x00],
        [0x02, 0x00, 0x00, 0x00],
        [0x04, 0x00, 0x00, 0x00],
        [0x08, 0x00, 0x00, 0x00],
        [0x10, 0x00, 0x00, 0x00],
        [0x20, 0x00, 0x00, 0x00],
        [0x40, 0x00, 0x00, 0x00],
        [0x80, 0x00, 0x00, 0x00],
        [0x1B, 0x00, 0x00, 0x00],
        [0x36, 0x00, 0x00, 0x00],
    ]
    
  

    w = [list(key[4 * i: 4 * (i + 1)]) for i in range(Nk)]
    
    for i in range(Nk, Nb * (Nr + 1)):
        temp = w[i - 1][:]  # Ensure a copy is made
        
        if i % Nk == 0:
            temp = [a ^ b for a, b in zip(sub_word(rot_word(temp)), RCON[i // Nk - 1])]
        elif Nk > 6 and i % Nk == 4:
            temp = sub_word(temp)
        
        w.append([a ^ b for a, b in zip(w[i - Nk], temp)])
    
    return w

In [5]:
def key_expansion(key):
    """Expand the cipher key into the key schedule."""
    RCON = [
        0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36
    ]
    
    key_symbols = [ord(symbol) for symbol in key]
    key_schedule = [[key_symbols[i + j * 4] for i in range(4)] for j in range(Nk)]
    
    for i in range(Nk, Nb * (Nr + 1)):
        temp = key_schedule[i - 1]
        
        if i % Nk == 0:
            temp = [SBOX[temp[(j + 1) % 4]] for j in range(4)]
            temp[0] ^= RCON[(i // Nk) - 1]
        
        key_schedule.append([key_schedule[i - Nk][j] ^ temp[j] for j in range(4)])
    
    return [key_schedule[4 * i:4 * (i + 1)] for i in range(len(key_schedule) // 4)]


In [6]:
def sub_bytes(state):
    return [[SBOX[byte] for byte in row] for row in state]


def shift_rows(state):
    return [
        state[0],
        state[1][1:] + state[1][:1],
        state[2][2:] + state[2][:2],
        state[3][3:] + state[3][:3]
    ]


def mix_columns(state):
    def gmul(a, b):
        p = 0
        for _ in range(8):
            if b & 1:
                p ^= a
            hi_bit_set = a & 0x80
            a = (a << 1) & 0xFF
            if hi_bit_set:
                a ^= 0x1B
            b >>= 1
        return p

    for i in range(4):
        a = state[i]
        state[i] = [
            gmul(a[0], 2) ^ gmul(a[1], 3) ^ gmul(a[2], 1) ^ gmul(a[3], 1),
            gmul(a[0], 1) ^ gmul(a[1], 2) ^ gmul(a[2], 3) ^ gmul(a[3], 1),
            gmul(a[0], 1) ^ gmul(a[1], 1) ^ gmul(a[2], 2) ^ gmul(a[3], 3),
            gmul(a[0], 3) ^ gmul(a[1], 1) ^ gmul(a[2], 1) ^ gmul(a[3], 2)
        ]

    return state


def add_round_key(state, round_key):
    return [[state[row][col] ^ round_key[row][col] for col in range(4)] for row in range(4)]


def cipher(plaintext_block, round_keys):
    state = [[plaintext_block[row + 4 * col] for col in range(4)] for row in range(4)]

    state = add_round_key(state, round_keys[0])

    for round_ in range(1, Nr):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, round_keys[round_])

    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, round_keys[Nr])

    return [state[row][col] for col in range(4) for row in range(4)]




#main function called to initiate encryption process
def aes_encrypt(plaintext, key):
    #128 bit key (16 bytes = 128 bits)
    assert len(key) == 16, "Key must be 16 bytes for AES-128."
    
    #128 bit blocks (16 bytes = 128 bits)
    plaintext = plaintext.ljust(16, '\x00')  # Padding block if necessary
    plaintext_block = [ord(char) for char in plaintext]
    
    
    round_keys = key_expansion(key)

    ciphertext_block = cipher(plaintext_block, round_keys)

    return ''.join(f'{x:02x}' for x in ciphertext_block)


In [7]:
# Route to serve static CSS files
@app.route('/static/<filename>')
def serve_static(filename):
    return static_file(filename, root='./static')

# Route to render the HTML page
@app.get('/')
def index():
    return template("index", strings=strings)

#Accept input & send to encryption
@app.post('/encrypt')
def encrypt():
    global next_id

    # Get user input
    input_value = request.forms.get('input')

    # Ensure input is valid
    if not input_value:
        return "<p>Error: Text input is required.</p><a href='/'>Go Back</a>"

    # Encrypt the input
    key = "thisisakey123456"
    input_encrypt = aes_encrypt(input_value, key)

    # Store the encrypted result
    strings[next_id] = {"id": next_id, "value": input_value, "encrypt": input_encrypt}
    next_id += 1

    # Redirect back to the landing page
    redirect('/')
    

In [ ]:
# Run the app
if __name__ == '__main__':
    run(app, host='localhost', port=8080, debug=True)

Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [04/Mar/2025 12:05:41] "GET / HTTP/1.1" 200 826
127.0.0.1 - - [04/Mar/2025 12:05:41] "GET /static/style.css HTTP/1.1" 200 517
127.0.0.1 - - [04/Mar/2025 12:05:41] "GET /favicon.ico HTTP/1.1" 404 748
127.0.0.1 - - [04/Mar/2025 12:05:45] "POST /encrypt HTTP/1.1" 303 0
127.0.0.1 - - [04/Mar/2025 12:05:45] "GET / HTTP/1.1" 200 904
